# Prognose Festgeldabschluss (Bank Marketing)

**Ziel:** Vorhersage, ob ein Kunde ein Festgeld abschließt (`y`: yes/no) auf Basis des UCI Bank-Marketing-Datensatzes.

**Inhalt:**
1. Setup und Konfiguration
2. Explorative Datenanalyse (EDA)
3. Baseline + erstes Modell
4. Evaluation & Interpretation
5. Management Summary
6. Weiterführende Fragen

## 1. Setup und Konfiguration

### Reproduzierbarkeit

Wir verwenden einen festen Zufalls-Seed (`RANDOM_STATE`), damit bei jedem Durchlauf die gleichen Ergebnisse herauskommen.

In [ ]:
import numpy as np
import pandas as pd

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE) # Reproducability

# Pandas view settings for better readability in the notebook
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

print("Setup OK, RANDOM_STATE =", RANDOM_STATE)

Setup OK, RANDOM_STATE = 42


### Auswahl Datensatz
Ich habe mich für den Datensatz *"bank-additional-full.csv"* entschieden. Dieser hat den Vorteil, dass er zusätzliche Parameter beinhaltet, die unser Modell präziser machen können. Parameter entfernen können wir immer noch.
Den Auszug mit zufällig ausgewählten Daten *"bank-additional.csv"* ignoriere ich, da ich einen time-based split nutze (s.u.).

In [ ]:
from pathlib import Path

# Load dataset
df_raw = pd.read_csv(Path("bank-additional-full.csv"), sep=";")
df_work = df_raw.copy()

print("Shape:", df_work.shape)

df_work.head(3)

Shape: (41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## 2. Schnelle Explorative Datenanalyse (EDA)

Vor dem Training gucke ich mir kurz an,

1) wie die Zielvariable verteilt ist (Imbalance)

2) ob irgendwelche Werte fehlen (kurzer Check, ob die Angaben in der Beschreibung korrekt sind)

3) Welche Datentypen die Variablen haben

In [ ]:
# Check distribution of target variable "y" (whether the client subscribed to a term deposit or not)
TARGET_COL = "y"
y_counts = df_work[TARGET_COL].value_counts(dropna=False)
y_share = df_work[TARGET_COL].value_counts(normalize=True, dropna=False)
display(y_counts)
display(y_share.rename("share"))

# Check for missing values in the dataset
missing_share = df_work.isna().mean().sort_values(ascending=False)
display(missing_share[missing_share > 0].rename("missing_share"))

# Visualize the datatypes of the columns in the dataset
display(df_work.dtypes)

y
no     36548
yes     4640
Name: count, dtype: int64

y
no     0.887346
yes    0.112654
Name: share, dtype: float64

Series([], Name: missing_share, dtype: float64)

age                 int64
job                   str
marital               str
education             str
default               str
housing               str
loan                  str
contact               str
month                 str
day_of_week           str
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome              str
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                     str
dtype: object

Ich sehe, dass wir eine Erfolgsquote bei den Anrufen von circa 11,2 % haben. Es liegt also eine Imbalance vor. Daher werde ich eine Baseline ziehen (immer "no") um zu sehen, ob mein Modell tatsächlich besser als dieses triviale Modell ist. Außerdem nutze ich diese Info für die Wahl meines KPIs in Kapitel 4 und bei der Konfiguration meines Modells.

Es fehlen wie beschrieben keine Daten.

Aufgrund der Datentypen weiß ich, dass ich verschiedene Preprocessings (Standard Scaler für die numerischen, One-Hot-Encoding für die kategorischen/str Datentypen) anwenden werde.

## 3. Baseline + erstes Modell

### Aufteilen der Daten in Train und Test (zeitbasierter Split)

Ich verwende einen zeitbasierten Split, d. h. das Modell wird auf einem früheren Zeitraum trainiert und auf einem späteren Zeitraum getestet.

Hinweis: Der Full-Datensatz *"bank-additional-full.csv"* ist chronologisch sortiert (Mai 2008 bis Nov 2010). Daher ist ein zeitbasierter Split hier plausibel.
Das 10%-Dataset *"bank-additional.csv"* ist hingegen eine zufällige Teilmenge und eignet sich primär für schnellere Experimente, nicht für zeitbasierte Evaluationsaussagen.

Den Parameter "duration", also die Länge des Telefonats, entferne ich gezielt aus den Trainingsdaten. Er ist nicht zielführend für unsere Prognose. Die Prognose soll dazu dienen, die richtigen Kandidaten zu ermitteln, mit denen ein Telefonat geführt wird, was aussichtsreiche Abschlusschancen hat.

In [ ]:
from sklearn.metrics import recall_score, confusion_matrix

# Drop duration as it is only known after the call.
LEAKAGE_COLS = ["duration"]
df_model = df_work.drop(columns=LEAKAGE_COLS).copy()

x = df_model.drop(columns=[TARGET_COL])
y = df_model[TARGET_COL]

# Time-based split: train on earlier rows, test on later rows.
test_fraction = 0.2 # Split 80% train, 20% test
split_idx = int((1 - test_fraction) * len(df_model))

x_train, x_test = x.iloc[:split_idx], x.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train:", x_train.shape, ", Test:", x_test.shape, ", Total:", x.shape)
display(y_train.value_counts(normalize=True).rename("train_share"))
display(y_test.value_counts(normalize=True).rename("test_share"))
display(y.value_counts(normalize=True).rename("full_share"))

# Plot success rate of phone calls over time
import plotly.express as px

# y as binary (1 for yes, 0 for no)
y_bin = (df_work["y"] == "yes").astype(int)

# Rolling mean over index
window = 1000
rolling_rate = y_bin.rolling(window=window, min_periods=window).mean()

plot_df = pd.DataFrame({
    "index": np.arange(len(df_work)),
    "rolling_yes_rate": rolling_rate
}).dropna()

fig = px.line(
    plot_df, x="index", y="rolling_yes_rate",
    title=f"Rolling subscription rate over time (window={window})",
    labels={"rolling_yes_rate": "share of 'yes'", "index": "chronological index"}
)
fig.update_layout(template="plotly_white")
fig.show()

Train: (32950, 19) , Test: (8238, 19) , Total: (41188, 19)


y
no     0.936267
yes    0.063733
Name: train_share, dtype: float64

y
no     0.691673
yes    0.308327
Name: test_share, dtype: float64

y
no     0.887346
yes    0.112654
Name: full_share, dtype: float64

Wir erkennen, dass in den Testdaten deutlich mehr positive Abschlüsse zu verzeichnen sind, als in den Trainingsdaten. Hier spielen, wie man später bei der Auswertung sieht, wahrscheinlich makroökonomische Ursachen (euribor) eine größere Rolle. Auf jeden Fall sehen wir eine Zeitabhängigkeit. Das akzeptieren wir in unserem Fall.
Wir werden später sehen, wie gut unser Modell trotz dieser unterschiedlichen Voraussetzungen ist. Auch in der Zukunft wird es Zeiten mit mehr Abschlüssen und Zeiten mit weniger Abschlüssen geben.

### KPIs
Bei der Bestimmung geeigneter Zielgrößen möchte ich mich auf zwei wichtige Punkte konzentrieren:
- einerseits möglichst wenige Personen zu übersehen, die das Modell fälschlicherweise als "no" einordnet, obwohl sie einen Vertrag abgeschlossen hätten. Hierbei muss aber berücksichtigt werden, ob es ausreichend potentielle Kunden gibt – davon hängt ab, wie schlimm ein übersehener Kunde tatsächlich ist.
- andererseits möglichst genau die Personen zu ermitteln, bei denen ein Abschluss sehr wahrscheinlich erfolgreich ist und für die es sich lohnt, die Zeit in ein oder mehrere Telefonate zu investieren.

Daraus ergeben sich zwei KPIs:

1. **Recall** – Wie viele der tatsächlich positiven Fälle finden wir?

   Recall = $\frac{TP}{TP + FN}$

   → Ziel: Möglichst wenige potentielle Kunden übersehen (FN minimieren).

2. **Precision** – Wie viele unserer Anrufe führen tatsächlich zum Abschluss?

   Precision = $\frac{TP}{TP + FP}$

   → Ziel: Möglichst wenig Zeit in erfolglose Anrufe investieren (FP minimieren).

Ich nutze den **Recall als primäre Metrik**, weil ich zunächst sicherstellen will, dass wir möglichst wenige Abschlüsse verpassen. Die **Precision** (Erfolgsquote pro Anruf) ziehe ich als zweite Kenngröße heran, um den praktischen Nutzen im Vertrieb bewerten zu können.

### Baseline-Modell

Bevor ich ein "echtes" Modell trainiere, etabliere ich eine **Baseline** als Referenz.
Eine einfache Baseline ist ein Klassifikator, der immer die häufigste Klasse vorhersagt (typischerweise "no").

Das ist wichtig, weil bei unausgeglichenen Klassen, wie in diesem Beispiel ("yes"/Abschluss ist selten), ein naiver Ansatz schnell "gute" Accuracy haben kann, aber fachlich wenig bringt.

In [65]:
from sklearn.dummy import DummyClassifier

# Baseline: always predict the most frequent class (typically "no").
baseline = DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE)
baseline.fit(x_train, y_train)

y_pred = baseline.predict(x_test)

def calculate_recall(y_true, y_pred, pos_label="yes"):
    # Recall focuses on capturing actual "yes" cases: TP / (TP + FN)
    recall = recall_score(y_true, y_pred, pos_label=pos_label)

    cm = confusion_matrix(y_true, y_pred, labels=["no", "yes"])
    tn, fp, fn, tp = cm.ravel()

    return recall, tp, fn, fp, tn

recall, tp, fn, fp, tn = calculate_recall(y_test, y_pred)

print("Baseline recall (pos='yes'):", round(recall, 4))
print(f"Confusion matrix counts: TP={tp}, FN={fn}, FP={fp}, TN={tn}")

Baseline recall (pos='yes'): 0.0
Confusion matrix counts: TP=0, FN=2540, FP=0, TN=5698


Die Baseline zeigt, dass ein einfaches Modell aufgrund der Imbalance nicht erfolgreich ist. Das umgekehrte Modell "immer yes" wäre das gleiche, wie einfach alle Personen anzurufen (Status Quo der Datengrundlage). Dieses nehmen wir für spätere Vergleiche.

### Erstes Modell: Linear SVC (Support Vector Machine)

Als erstes echtes Modell nutze ich eine **lineare SVM** (`LinearSVC`).
Begründung:
- Der [scikit-learn Estimator-Guide](https://scikit-learn.org/stable/machine_learning_map.html) nennt Linear SVC als sinnvollen Startpunkt für Klassifikation bei Datensätzen <100k Samples.
- Die Daten enthalten gemischte Feature-Typen (numerisch + kategorisch). Daher verwende ich eine Preprocessing-Pipeline:
  - **kategorisch:** One-Hot-Encoding
  - **numerisch:** Standardisierung (Feature Scaling)
- Da `y=yes` deutlich seltener ist (Imbalance), nutze ich **class_weight="balanced"**, um die Minderheitsklasse stärker zu gewichten.


In [66]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Identify column groups
numeric_features = x_train.select_dtypes(include=["number"]).columns.tolist()
categorical_features = x_train.select_dtypes(exclude=["number"]).columns.tolist()

print("Num features:", len(numeric_features))
print("Cat features:", len(categorical_features))

# Preprocessing: scale numeric, one-hot encode categorical
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),  # scaling helps SVM behave well on numeric features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features), # one-hot encoding for categorical features
    ],
    remainder="drop"
) # ColumnTransformer applies per-column transformations

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

# Linear SVM with class balancing for imbalanced target
clf = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LinearSVC(class_weight="balanced", random_state=RANDOM_STATE, max_iter=5000)), # raise max_iter to ensure convergence
    ]
)

clf.fit(x_train, y_train)
print("Model fitted.")

y_pred = clf.predict(x_test)
recall, tp, fn, fp, tn = calculate_recall(y_test, y_pred)

print("Recall (pos='yes'):", round(recall, 4))
print(f"TP={tp}, FN={fn}, FP={fp}, TN={tn}")

original_call_success = df_raw["y"].value_counts(normalize=True)["yes"]
model_call_success = tp/(tp + fp) # Precision: success rate among all calls that the model predicted as "yes"

print("Original call success rate in dataset (baseline):", round(original_call_success, 4))
print("Model call success rate:", round(model_call_success, 4))

Num features: 9
Cat features: 10
Model fitted.
Recall (pos='yes'): 0.6854
TP=1741, FN=799, FP=3216, TN=2482
Original call success rate in dataset (baseline): 0.1127
Model call success rate: 0.3512


## 4. Evaluation & Interpretation

Anhand der obigen Berechnung sehen wir, dass der Recall 68,54 % beträgt. Das bedeutet, dass 68,54 % der Kunden angerufen worden wären, die tatsächlich auch einen Vertrag unterschrieben haben.
Gleichzeitig wären 35,1 % unserer Anrufe erfolgreich (also mit Vertragsabschluss) gewesen, während bei dem kompletten Datensatz (ohne unser Modell) nur 11,2 % der Anrufe zu einem erfolgreichen Abschluss geführt haben.

Zur weiteren Auswertung gucken wir uns die Confusion Matrix genauer an:

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=["no", "yes"])
cm_df = pd.DataFrame(cm, index=["actual_no", "actual_yes"], columns=["pred_no", "pred_yes"])

fig = px.imshow(
    cm_df,
    text_auto=True,
    color_continuous_scale="Blues",
    title="Confusion Matrix (LinearSVC, time-based split)"
)
fig.update_layout(xaxis_title="Predicted", yaxis_title="Actual", template="plotly_white")
fig.show()

Die Confusion Matrix zeigt transparent, welche Fehlerarten entstehen (insb. FN vs. FP). Die absoluten Zahlen beziehen sich auf unseren Test-Datensatz und weichen daher von der Gesamtverteilung ab.

Wir sehen, dass wir zwar 1741 Vertragsabschlüsse generiert hätten.
Dafür hätten wir aber auch Zeit aufgewendet, um 3216 Kunden anzurufen, die am Ende keinen Vertrag abschließen.
Gleichzeitig hätten wir in unserer Liste 799 Kunden nicht angerufen, mit denen wir zu einem erfolgreichen Abschluss gekommen wären.

### Optimierung der Anrufe

Ich nutze nun das bestehende Modell und ändere den Schwellwert (der standardmäßig 0 ist) der numerischen Vorhersage, ab wann wir mit einem Vertragsabschluss rechnen. Das Ganze visualisieren wir und diskutieren im Anschluss den Tradeoff:

In [ ]:
scores = clf.decision_function(x_test)  # higher => more "yes"

# Sweep thresholds across score range
thresholds = np.quantile(scores, np.linspace(0.01, 0.99, 60))

rows = []
for t in thresholds:
    pred_t = np.where(scores >= t, "yes", "no")
    cm_t = confusion_matrix(y_test, pred_t, labels=["no", "yes"])  # [[TN, FP],[FN, TP]]
    tn, fp, fn, tp = cm_t.ravel()
    recall_yes = tp / (tp + fn) if (tp + fn) else np.nan
    precision_yes = tp / (tp + fp) if (tp + fp) else np.nan
    rows.append({"threshold": t, "recall": recall_yes, "precision": precision_yes, "FP": fp, "FN": fn, "TP": tp})

df_thr = pd.DataFrame(rows)

fig = px.line(
    df_thr,
    x="threshold",
    y=["recall", "precision"],
    title="Recall/Precision vs Threshold (decision_function)",
    labels={"value": "metric", "threshold": "threshold"}
)
fig.update_layout(template="plotly_white")
fig.show()

fig2 = px.line(
    df_thr,
    x="threshold",
    y=["FN", "FP", "TP"],
    title="FN/FP vs Threshold",
    labels={"value": "count", "threshold": "threshold"}
)
fig2.update_layout(template="plotly_white")
fig2.show()

#### Recall/Precision
Im oberen Plot sehen wir, wie der Recall naturgemäß bei steigendem Grenzwert sinkt. Wir finden also weniger der Kunden, die einen Vertrag abschließen würden. Im Gegenzug würden wir natürlich auch deutlich weniger Personen anrufen. Unsere Anrufe werden also effizienter, aber wir übersehen auch potentielle Kunden.

Die Precision nimmt mit zunehmendem Grenzwert zu. Die Wahrscheinlichkeit, dass ein Anrufer am Ende auch einen Vertrag abschließt, wird also höher. Im Gegenzug übersehen wir wie oben erläutert potentielle Kunden.

#### True Positives, False Negatives und False Positives
In der unteren Grafik sehen wir, dass bei dem aktuellen Grenzwert von 0 das Bild identisch mit der oben erläuterten Confusion Matrix ist.

Wir müssen uns entscheiden, was genau das Ziel ist:
1. Wir wollen unseren Telefonprozess effizienter machen. Da unsere Kundenanzahl begrenzt ist, wollen wir so wenige potentielle Vertragsabschlüsse übersehen wie möglich. Dann müssen wir den Grenzwert eher herabsetzen. Der Wert 0 scheint bereits ein relativ guter Trade-Off zu sein.
2. Wir haben einen riesigen Kundenpool und schaffen es sowieso nicht, alle Kunden anzurufen. Die Zeit unserer Kundenberater ist limitiert und wir wollen möglichst viele Abschlüsse bei einer konstanten Anzahl von Anrufen erreichen. Dann erhöhen wir den Grenzwert. Der Schnittpunkt der drei Linien bei einem Wert von circa 0,2 scheint hier ein vernünftiger Kompromiss zu sein. Wir haben ungefähr genau so viele False Positives wie True Positives, circa 50 % der Anrufe führen zu einem Vertragsabschluss. Dieses Verhältnis bleibt auch bei steigendem Grenzwert relativ konstant, sodass wir damit keinen weiteren Benefit erzielen.

### Einflussgrößen
Im Folgenden gucken wir uns nochmal die Faktoren an, die unser Modell als positiv und als negativ für einen potentiellen Abschluss eines Festgeldvertrages ermittelt hat:

In [ ]:
# Access fitted components
preprocess = clf.named_steps["preprocess"]
model = clf.named_steps["model"]

# Get feature names after preprocessing
feature_names = preprocess.get_feature_names_out()
coefs = model.coef_.ravel()

coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs})
coef_df["abs_coef"] = coef_df["coef"].abs()

top_pos = coef_df.sort_values("coef", ascending=False).head(15)
top_neg = coef_df.sort_values("coef", ascending=True).head(15)

fig_pos = px.bar(top_pos, x="coef", y="feature", orientation="h",
                 title="Top positive drivers (towards 'yes')", labels={"coef": "coefficient"})
fig_pos.update_layout(template="plotly_white", yaxis={"categoryorder": "total ascending"})
fig_pos.show()

fig_neg = px.bar(top_neg, x="coef", y="feature", orientation="h",
                 title="Top negative drivers (towards 'no')", labels={"coef": "coefficient"})
fig_neg.update_layout(template="plotly_white", yaxis={"categoryorder": "total ascending"})
fig_neg.show()

**Makro-/Umfeldfaktoren:**
Der Referenzzinssatz Euribor 3m ist ein starker positiver Treiber. Das ist plausibel, da Konditionen und Abschlussbereitschaft typischerweise vom Zins- und Marktumfeld beeinflusst werden. Gleichzeitig ist Euribor nicht kundenindividuell, sondern zeitabhängig: Er hilft eher bei der Einordnung *wann* Kampagnen erfolgreicher sind, aber weniger bei der Auswahl *welche* Einzelkunden man anspricht.

**Kontakt-/Kampagnenkontext:**
Variablen wie *Monat* oder *Kontaktkanal* erscheinen ebenfalls als Treiber. Diese Merkmale sind teilweise steuerbar (z.B. Kanalwahl) und können gleichzeitig Saison-/Kampagneneffekte widerspiegeln. Da der Datensatz chronologisch sortiert ist, können solche Variablen auch Zeitphasen markieren.

**Kundenmerkmale:**
Einige Kundenmerkmale wie Beruf oder Bildung helfen dem Modell, Kunden mit höherer Abschlusswahrscheinlichkeit von anderen zu unterscheiden. Diese Merkmale sind für die Zielgruppenauswahl nützlicher als reine Umfeldfaktoren (z.B. Euribor), weil sie sich pro Kunde unterscheiden. Wichtig ist nur: Da Kategorien im Modell in mehrere 0/1-Spalten umgewandelt werden (One-Hot-Encoding) und Zahlen skaliert werden, sollten die Koeffizienten als Richtungshinweis verstanden werden, nicht als exaktes Mapping.

**Take-away:**
Das Modell deutet darauf hin, dass sowohl Umfeldfaktoren (Makro/Timing) als auch kundenbezogene Merkmale Einfluss haben. Für operative Selektion sind vor allem kundenbezogene Features und steuerbare Kontaktmerkmale interessant; Makrovariablen liefern eher Hinweise zur Kampagnenplanung bzw. zum Timing.

## 5. Management Summary

Im Folgenden fasse ich die Ergebnisse kompakt und ohne technische Details zusammen – gedacht als Entscheidungsvorlage für Stakeholder.

### Ausgangslage
Wir haben eine Telefonkampagne untersucht, mit der Kunden für Festgeldprodukte gewonnen werden sollen. Im Datensatz (41.188 Kundenkontakte, Mai 2008 bis November 2010) führten **nur ca. 11 % der Anrufe zu einem Abschluss** – rund 9 von 10 Anrufen blieben also ohne Ergebnis.

### Was wurde entwickelt?
Ein datengetriebenes Prognosemodell, das auf Basis vorhandener Kundenmerkmale, Kampagnendaten und makroökonomischer Indikatoren **vor dem Anruf** einschätzt, ob ein Kunde wahrscheinlich ein Festgeld abschließen wird.

Wichtig: Die Gesprächsdauer (`duration`) fließt bewusst **nicht** in das Modell ein, da sie erst nach dem Telefonat bekannt ist. Nur so bleibt die Prognose praxistauglich.

### Kernergebnisse

| Kennzahl | Ohne Modell | Mit Modell |
|---|---|---|
| **Erfolgsquote pro Anruf** | ~11 % | ~35 % |
| **Erkannte Abschlüsse (Recall)** | 100 % (alle anrufen) | ~69 % |
| **Anrufvolumen (Testdaten)** | 8.238 | 4.957 (−40 %) |

- **3× höhere Erfolgsquote:** Statt 11 % führen nun rund 35 % der Anrufe zu einem Vertragsabschluss.
- **40 % weniger Anrufe nötig:** Das Modell reduziert das Anrufvolumen erheblich, bei gleichzeitig deutlich höherer Trefferquote.
- **Trade-off:** Circa 31 % der potentiellen Abschlüsse (799 von 2.540 im Testzeitraum) werden vom Modell nicht erkannt. Ob das akzeptabel ist, hängt von der Größe des Kundenpools und den verfügbaren Beraterkapazitäten ab.

### Stellschrauben
Der Schwellwert des Modells kann je nach Geschäftsziel angepasst werden:
- **Maximale Reichweite:** Schwellwert senken → mehr potentielle Kunden werden erkannt, dafür sinkt die Effizienz pro Anruf.
- **Maximale Effizienz:** Schwellwert erhöhen → bis zu ~50 % Erfolgsquote pro Anruf möglich, allerdings werden dann mehr potentielle Kunden übersehen.

### Wichtigste Einflussfaktoren
| Faktor | Bedeutung |
|---|---|
| **Euribor 3M / Makroumfeld** | Starker Treiber – zeigt, *wann* Kampagnen besonders erfolgreich sind (Timing). |
| **Kontaktkanal & Monat** | Steuerbare Variablen – deuten auf optimale Kanäle und Zeitfenster hin. |
| **Kundenmerkmale (Beruf, Bildung)** | Helfen bei der Selektion, *welche* Kunden priorisiert angesprochen werden sollten. |

### Empfehlung
Das Modell kann den bestehenden Kampagnenprozess sinnvoll ergänzen: Anstatt alle Kunden gleichmäßig zu kontaktieren, ermöglicht es eine **gezielte Priorisierung**. Die frei werdenden Kapazitäten können für intensivere Kundenbetreuung oder weitere Kampagnen genutzt werden. Die genaue Kalibrierung (Schwellwert) sollte gemeinsam mit dem Vertrieb festgelegt werden – abhängig von Kundenpool-Größe und Beraterkapazität.

## 6. Weiterführende Fragen

### Würde ich für die Voraussage noch weitere Datenpunkte in Erwägung ziehen?

Ja, auf jeden Fall. Im Datensatz stecken vor allem Kampagnen- und Makrodaten, aber kaum **individuelle Finanzdaten**. Die hätte eine Bank aber typischerweise griffbereit – und ich halte sie für besonders vielversprechend:

- **Saldo Girokonto:** Wer viel frei verfügbares Guthaben hat, ist der naheliegendste Festgeld-Kandidat. Das Geld liegt ja quasi schon bereit.
- **Bestehende Festgeldverträge inkl. Laufzeitende:** Kunden, deren Festgeld demnächst ausläuft, stehen aktiv vor einer Wiederanlage-Entscheidung. Den Zeitpunkt kurz vor Fälligkeit würde ich gezielt für die Ansprache nutzen.
- **Gehalt / monatliches Nettoeinkommen:** In Kombination mit dem Girokonto-Saldo kann ich abschätzen, ob jemand überhaupt Mittel für ein Festgeld hätte.
- **Sparquote / Transaktionsmuster:** Kunden, die regelmäßig Überschüsse aufbauen (z. B. Daueraufträge auf Sparkonten), zeigen bereits aktives Sparverhalten – ein starkes Signal.
- **Kundenalter der Bankbeziehung:** Langjährige Kunden haben eventuell mehr Vertrauen und sind eher bereit, weitere Produkte abzuschließen.

Diese Daten liegen im Kernbanksystem bzw. CRM typischerweise vor und müssten lediglich datenschutzkonform angebunden werden.

### Können die Erkenntnisse auch für andere Bereiche der Bank verwendet werden?

Ja, ich denke schon. Der grundsätzliche Ansatz – auf Basis von Kundendaten vorhersagen, wer ein bestimmtes Produkt abschließt – lässt sich auf andere Bereiche übertragen:

**1. Kreditvergabe (Cross-Selling)**
Das gleiche Modellkonzept kann ich nutzen, um vorherzusagen, welche Kunden einen Kredit aufnehmen würden. Spannend ist hier, dass einige Faktoren vermutlich **gegenläufig** wirken: Ein hoher Kontostand spricht eher gegen Kreditbedarf, während er für Festgeld spricht. Die Zielvariable wäre dann `y` = „Kredit abgeschlossen", und die Feature-Importance wird sich entsprechend verschieben.

**2. Kampagnen-Timing fürs Marketing**
Unsere Analyse hat gezeigt, dass Makrofaktoren (insb. Euribor) einen starken Einfluss auf die Abschlusswahrscheinlichkeit haben. Das kann das Marketing nutzen, um Kampagnen gezielt in Phasen zu planen, in denen das Marktumfeld günstig ist – egal für welches Produkt.

**3. Kundenabwanderung (Churn)**
Mit einer angepassten Zielvariable (`y` = „Kunde hat Bank verlassen") und ergänzenden Features (z. B. sinkende Transaktionsaktivität, aufgelöste Produkte) kann ich das gleiche Framework einsetzen, um abwanderungsgefährdete Kunden frühzeitig zu erkennen.

### Muss die Problemstellung angepasst werden? Wie würde ich vorgehen?

Ja, für jeden neuen Anwendungsfall müssen im Wesentlichen drei Dinge angepasst werden:

1. **Zielvariable neu definieren** – z. B. „Kredit abgeschlossen" oder „Kunde abgewandert" statt „Festgeld abgeschlossen".
2. **Features prüfen und erweitern** – Nicht alle Features aus dem Festgeld-Modell sind für andere Produkte relevant. Dafür kommen neue hinzu (z. B. Kredithistorie für ein Kreditmodell).
3. **Metrik und Schwellwert geschäftsspezifisch setzen** – Bei Churn wäre z. B. der Recall noch wichtiger (keinen gefährdeten Kunden übersehen), bei Kredit-Cross-Selling eher die Precision (Berater-Zeit nicht verschwenden).

Die technische Pipeline (Preprocessing, Train/Test-Split, Modelltraining, Evaluation) bleibt dabei weitgehend identisch und kann als Vorlage wiederverwendet werden. In einer robusten Architektur könnte sie sogar parametrisiert werden, sodass auch Kollegen ohne ML-Erfahrung damit arbeiten können.